## Clone repos from github

In [1]:
%%capture
!git clone https://github.com/hukenovs/hagrid.git
# or mirror link:
%cd hagrid
# Install requirements
%pip install -r requirements.txt --user
%cd ..
# !wget https://raw.githubusercontent.com/shitkov/signature_detector/main/dataset.zip
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt
# !unzip /content/dataset.zip -d /content/yolov5/
%cd ..

In [2]:
# !rm hagrid_data -r

In [3]:
# !python hagrid/download.py --save_path /kaggle/temp/hagrid_data \
#                    --train \
#                    --targets call \
#                    --test \
#                   #  --subset \
#                    --annotations \
#                    --dataset


In [4]:
##### Subsample
# !mkdir 'hagrid_data/subsample/subsample'
# !unzip -q 'hagrid_data/subsample/subsample.zip' -d 'hagrid_data/subsample/subsample'
# !unzip -q 'hagrid_data/subsample/ann_subsample.zip' -d 'hagrid_data/subsample/ann_subsample'

## YOLOv5 habr

In [5]:
%%capture
# !wget https://raw.githubusercontent.com/shitkov/signature_detector/main/dataset.zip
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt
# !unzip /content/dataset.zip -d /content/yolov5/
%cd ..

## Preprocessing annotation files to YOLOv5 format

In [6]:
import json
import os

YOLO_ANN_TRAIN_JSON_PATH = '/kaggle/input/hagrid-call-test/ann_train_val/ann_train_val/call.json'
YOLO_ANN_TEST_JSON_PATH  = '/kaggle/input/hagrid-call-test/ann_test/ann_test/call.json'
YOLO_ANN_TRAIN_TXTS_PATH = '/kaggle/working/labels_train'
YOLO_ANN_TEST_TXTS_PATH  = '/kaggle/working/labels_test'
HAGRID_PHOTO_TRAIN_PATH  = '/kaggle/input/hagrid-call-test/train_val_call'
HAGRID_PHOTO_TEST_PATH   = '/kaggle/input/hagrid-call-test/test/call'

YOLO_ANN_FAKES_JSON_PATH = '/kaggle/input/hagrid-call-test/ann_test/ann_test'
YOLO_ANN_FAKES_TXTS_PATH = '/kaggle/working/labels_fakes'
HAGRID_PHOTO_FAKES_PATH  = '/kaggle/input/hagrid-call-test/test'


def preprocess_json2txt(json_path:str, txt_path:str, photo_path:str) -> None:
    dct={}
    with open(json_path) as f:
        d = json.load(f)
        for key in d:
            dct[key] = d[key]

    if not(os.path.isdir(txt_path)):
        os.mkdir(txt_path)

    filenames = []
    for (dirpath, dirnames, f) in os.walk(photo_path):
        filenames.extend(f)
        break

    for filename in filenames:
        name = filename.split('.')[0]
        with open(os.path.join(txt_path, name+'.txt'), 'w') as f:
            lines = []
            for i in range(len(dct[name]['bboxes'])):
                label = (1 if dct[name]['labels'][i]=='call' else 0)
                bbox = [dct[name]['bboxes'][i][0]+0.5*dct[name]['bboxes'][i][2],
                        dct[name]['bboxes'][i][1]+0.5*dct[name]['bboxes'][i][3],
                        dct[name]['bboxes'][i][2],
                        dct[name]['bboxes'][i][3]]
                lines.append(str(label) + ' ' + ' '.join(str(x) for x in bbox) + '\n')
            f.writelines(lines)
            
preprocess_json2txt(YOLO_ANN_TRAIN_JSON_PATH, YOLO_ANN_TRAIN_TXTS_PATH, HAGRID_PHOTO_TRAIN_PATH)
preprocess_json2txt(YOLO_ANN_TEST_JSON_PATH,  YOLO_ANN_TEST_TXTS_PATH,  HAGRID_PHOTO_TEST_PATH )

fakes_names=['dislike','fist','four','like','mute','ok','one','palm','peace','peace_inverted','rock',\
             'stop','stop_inverted','three','three2','two_up','two_up_inverted']

for name in fakes_names:
    preprocess_json2txt(YOLO_ANN_FAKES_JSON_PATH+'/'+name+'.json', YOLO_ANN_FAKES_TXTS_PATH,
                        HAGRID_PHOTO_FAKES_PATH+'/'+name)

In [7]:
# with open('./sig.yaml', 'w') as f:
#     f.write(f'train: {HAGRID_PHOTO_TRAIN_PATH}\n' + \
#             f'val: {HAGRID_PHOTO_TEST_PATH}\n' + '\n' + \
#             'nc: 2\n' + "names: ['call', 'no_gesture]")

In [8]:
### OBSOLETE
def create_dataset(dataset_path:str, img_train_path:str, label_train_path:str, valid_size:float,
                  random_state:int):
    np.random.seed(random_state)
    dirpath=''
    images = []
    for (dirpath_, dirpaths, filenames) in os.walk(img_train_path):
        images.extend(filenames)
        dirpath += dirpath_
        break
        
    images.sort()
    n = len(images)
    train_indices = np.arange(n)
    np.random.shuffle(train_indices)
    train_indices = train_indices[:n]
    mask = np.zeros(n, dtype=np.bool)
    for i in train_indices:
        mask[i]=1
    
    for i,image in enumerate(images):
        name = image.split('.')[0]
        dir_type = ('train' if i in mask[i]==1 else 'valid')
        exec(f'!cp -s {os.path.join(img_train_path, image)}' + \
             f'{os.path.join(dataset_path, "images", dir_type, image)}')
        txt = name+'.txt'
        exec(f'!cp -s {os.path.join(label_train_path, txt)}' + \
             f'{os.path.join(dataset_path, "labels", dir_type, txt)}')

In [9]:
# create_dataset('/kaggle/temp/dataset', HAGRID_PHOTO_TRAIN_PATH, YOLO_ANN_TRAIN_TXTS_PATH, valid_size=0.05, random_state=1)

In [10]:
!mkdir -p /kaggle/temp/dataset/images
!mkdir -p /kaggle/temp/dataset/labels
%cd ../temp
!cp -r -s '/kaggle/input/hagrid-call-test/train_val_call' './dataset/images'
!mv './dataset/images/train_val_call' './dataset/images/train'
!cp -r -s '/kaggle/input/hagrid-call-test/test/call'      './dataset/images'
!mv './dataset/images/call' './dataset/images/valid'
!cp -r -s '/kaggle/working/labels_train' './dataset/labels'
!mv './dataset/labels/labels_train' './dataset/labels/train'
!cp -r -s '/kaggle/working/labels_test' './dataset/labels'
!mv './dataset/labels/labels_test' './dataset/labels/valid'
%cd ../working

/kaggle/temp
/kaggle/working


In [11]:
from multiprocessing import Pool

def _scan_folder(dataset_path, dirs):
    lists = {}
    s1='images'
    for s2 in dirs:
        lists[s2] = []
        cur_path = os.path.join(dataset_path, s1, s2)
        for (dirpath, dirpaths, filenames) in os.walk(cur_path):
                for i in range(len(filenames)):
                    filenames[i] = filenames[i].split('.')[0]
                lists[s2].extend(sorted(filenames))
    return lists

def _trim_dataset_helper(dataset_path:str, s2:str, name:str):
    os.remove(os.path.join(dataset_path, 'images', s2, name + '.jpg'))
    os.remove(os.path.join(dataset_path, 'labels', s2, name + '.txt'))
    
def trim_dataset(dataset_path:str, n_train:int, n_valid:int, batch_index=0):
    dirs = [['images','labels'], ['train','valid']]
    lists = _scan_folder(dataset_path, dirs[1]) 
    
    for s2 in dirs[1]:
        n = (n_train if s2=='train' else n_valid)
        filenames = lists[s2].copy()
        _start = (batch_index * n) % (len(filenames))
        _end   = _start + n
        filenames = ((filenames[:_start] + filenames[_end:]) if _end <= len(filenames) else \
                      filenames[_end%len(filenames):_start])
            
        with Pool(4) as p:
            p.starmap(_trim_dataset_helper, [(dataset_path, s2, name) for name in filenames])
                
    lists = _scan_folder(dataset_path, dirs[1])
    return lists

In [12]:
def _copy_fakes_helper(dataset_path:str, image_path:str, label_path:str, filename:str, dir_type:str):
    filename = filename.split('.')[0]
    os.popen(f'cp -s {os.path.join(image_path, filename+".jpg")} {os.path.join(dataset_path, "images", dir_type)}')
    os.popen(f'cp -s {os.path.join(label_path, filename+".txt")} {os.path.join(dataset_path, "labels", dir_type)}') 
    
def copy_fakes(dataset_path:str, image_path:str, label_path:str, n_train:int,
               n_valid:int, fakes_names:list, batch_index=0):
    
    n_train = n_train//len(fakes_names)
    n_valid = n_valid//len(fakes_names)
    
    for name in fakes_names:
        dir_path = image_path+'/'+name
        filenames = sorted(os.listdir(dir_path))
        _start = (batch_index * (n_train + n_valid)) % (len(filenames))
        _end   = _start + (n_train + n_valid)
        filenames = (filenames[_start:_end] if _end <= len(filenames) else \
                     filenames[_start:] + filenames[:_end%len(filenames)])
        
        with Pool(4) as p:
            p.starmap(_copy_fakes_helper, [(dataset_path, dir_path, label_path, filename, 'train')\
                                           for filename in filenames[:n_train]])
        with Pool(4) as p:
            p.starmap(_copy_fakes_helper, [(dataset_path, dir_path, label_path, filename, 'valid')\
                                           for filename in filenames[n_train:n_train+n_valid]])
        
    lists = _scan_folder(dataset_path, ['train','valid'])
    return lists

In [13]:
# _scan_folder('/kaggle/temp/dataset', [['images','labels'], ['train','valid']])

## Trimming dataset

In [14]:
N_TRAIN = 5000
N_VALID = 500
BATCH_INDEX=1

dataset_files = trim_dataset('/kaggle/temp/dataset', N_TRAIN//2, N_VALID//2, BATCH_INDEX)
dataset_files = copy_fakes('/kaggle/temp/dataset', HAGRID_PHOTO_FAKES_PATH, YOLO_ANN_FAKES_TXTS_PATH,
                          N_TRAIN//2, N_VALID//2, fakes_names, BATCH_INDEX)
print(f"Batch index: {BATCH_INDEX}" + \
      f"Train dataset length: {len(dataset_files['train'])}\n" + \
      f"Valid dataset length: {len(dataset_files['valid'])}")

Batch index: 1Train dataset length: 4999
Valid dataset length: 488


In [15]:
with open('./sig.yaml', 'w') as f:
    f.write('train: /kaggle/temp/dataset/images/train/\n' + \
            'val: /kaggle/temp/dataset/images/valid/\n' + \
            '\n' + \
            'nc: 2\n' + \
            "names: ['no_gesture','call']\n")

In [16]:
!cat ./sig.yaml

train: /kaggle/temp/dataset/images/train/
val: /kaggle/temp/dataset/images/valid/

nc: 2
names: ['no_gesture','call']


In [17]:
# # !rm -r /kaggle/temp
# !mkdir -p /kaggle/temp/dataset/images
# !cp -r -s '/kaggle/input/hagrid-call-test/train_val_call' '/kaggle/temp/dataset/images'

In [18]:
# !ls /kaggle/temp/dataset

In [19]:
# !pip uninstall wandb -y

In [20]:
import torch
from IPython.display import Image, clear_output
from gc import collect
collect()

0

## Download weights

In [21]:
!wget https://github.com/SpryGorgon/YOLOv5/raw/main/yolov5s_1280_10.pt

--2022-10-09 09:06:09--  https://github.com/SpryGorgon/YOLOv5/raw/main/yolov5s_1280_10.pt
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/SpryGorgon/YOLOv5/main/yolov5s_1280_10.pt [following]
--2022-10-09 09:06:09--  https://raw.githubusercontent.com/SpryGorgon/YOLOv5/main/yolov5s_1280_10.pt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14624117 (14M) [application/octet-stream]
Saving to: ‘yolov5s_1280_10.pt’

yolov5s_1280_10.pt  100%[===================>]  13.95M  --.-KB/s    in 0.08s   

2022-10-09 09:06:10 (174 MB/s) - ‘yolov5s_1280_10.pt’ saved [14624117/14624117]



## Training YOLOv5

In [22]:
%cd yolov5
!python train.py --img 1280 --batch -1 --epochs 10 --data '../sig.yaml' --cfg yolov5s.yaml --weights ../yolov5s_1280_10.pt --cache disk --device 0
%cd ..

/kaggle/working/yolov5
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 
wandb: W&B disabled due to login timeout.
train: weights=../yolov5s_1280_10.pt, cfg=yolov5s.yaml, data=../sig.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=10, batch_size=-1, imgsz=1280, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=disk, image_weights=False, device=0, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.2-187-g5ef69ef Python-3.7.12 torch-1.11.0 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

hyperpar

In [23]:
# !tensorboard --logdir yolov5/runs/train

In [24]:
!cp './yolov5/runs/train/exp/weights/best.pt' -r '/kaggle/working/yolov5s_1280_20.pt'

## Validating 

In [25]:
%cd yolov5
!python val.py --weights './runs/train/exp/weights/best.pt' --data '../sig.yaml' --img 1280 --device 0
%cd ..

/kaggle/working/yolov5
val: data=../sig.yaml, weights=['./runs/train/exp/weights/best.pt'], batch_size=32, imgsz=1280, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=0, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.2-187-g5ef69ef Python-3.7.12 torch-1.11.0 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Fusing layers... 
YOLOv5s summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning '/kaggle/temp/dataset/labels/valid.cache' images and labels... 488
                 Class     Images  Instances          P          R      mAP50   
                   all        488        600      0.996      0.981      0.994      0.828
            no_gesture        488        350      0.992      0.963      0.994      0.817
                  call        488        250      0.999          1      0.995      0.839
Speed

In [26]:
# %cp '/yolov5/runs/train/exp/weights/best.pt' -r '/yolov5m_1024p_300.pt'

In [27]:
from PIL import Image
# img = Image.open('yolov5/runs/val/exp/val_batch2_pred.jpg')
# img.show()


# model = torch.hub.load('yolov5', 'custom', path='yolov5n_640_30.pt', source='local')
# imgs = ['/kaggle/temp/dataset/images/valid/' + x + '.jpg' for x in dataset_files['valid']]
# results = model(['/kaggle/temp/dataset/images/valid/e9524db2-1590-4c77-a084-ac217907e56f.jpg'])
# results.print()
# results.show()

In [28]:
# !ls /kaggle/temp/dataset/images/valid

## Detecting

In [29]:
%cd yolov5
!python detect.py --weights './runs/train/exp/weights/best.pt' --imgsz 1280 --device 0 --source /kaggle/temp/dataset/images/valid
%cd ..

/kaggle/working/yolov5
detect: weights=['./runs/train/exp/weights/best.pt'], source=/kaggle/temp/dataset/images/valid, data=data/coco128.yaml, imgsz=[1280, 1280], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=0, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v6.2-187-g5ef69ef Python-3.7.12 torch-1.11.0 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Fusing layers... 
YOLOv5s summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
image 1/488 /kaggle/temp/dataset/images/valid/193f9273-9590-4337-bbea-162dde922a58.jpg: 1280x960 1 call, 12.0ms
image 2/488 /kaggle/temp/dataset/images/valid/1955eef0-3514-4043-baa6-bb10d0eb0147.jpg: 1280x608 1 no_gesture, 1 call, 11.7ms
image 3/488 /kaggle/temp/dataset/images/valid/1957f93d-662

## Saving runs output to .tar

In [30]:
!tar -czf runs_output.zip './yolov5/runs'

In [31]:
# !ls ./yolov5/runs/detect/exp

In [32]:
# imgs = ['./yolov5/runs/detect/exp/' + x + '.jpg' for x in dataset_files['valid']]
# img = Image.open(imgs[0])
# img.show()


## Inference

In [33]:
# %cd yolov5
# !wget https://raw.githubusercontent.com/SpryGorgon/YOLOv5/main/predict.py
# !wget https://raw.githubusercontent.com/SpryGorgon/YOLOv5/main/helpers.py
# %cd ..

In [34]:
# from predict import predict
# from helpers import draw_bboxs, image_resizer
# import PIL
# from PIL import Image

In [35]:
# weights = 'yolov5n_640_30.pt'
# # image_path = '/content/test_multi.jpg'
# resized_image_save_path = '/content/resized_image.jpg'
# bboxs_image_save_path = '/content/predicted_bboxs.jpg'
# size = 1280